# Importando as bibliotecas necessárias

In [2]:
const faker = require('faker');
const mongoose = require('mongoose');
const _ = require('lodash');
const countryData = require('country-data');
mongoose.set('useCreateIndex', true);

Mongoose {
  connections: [
    NativeConnection {
      base: [Circular],
      collections: {},
      models: {},
      config: [Object],
      replica: false,
      options: null,
      otherDbs: [],
      relatedDbs: {},
      states: [Object: null prototype],
      _readyState: 0,
      _closeCalled: false,
      _hasOpened: false,
      plugins: [],
      id: 0,
      _listening: false
    }
  ],
  models: {},
  modelSchemas: {},
  options: {
    pluralization: true,
    useCreateIndex: true,
    [Symbol(mongoose:default)]: true
  },
  _pluralize: [Function: pluralize],
  Schema: [Function: Schema] {
    reserved: [Object: null prototype] {
      validate: 1,
      toObject: 1,
      schema: 1,
      save: 1,
      remove: 1,
      populated: 1,
      isNew: 1,
      isModified: 1,
      init: 1,
      get: 1,
      errors: 1,
      collection: 1,
      removeListener: 1,
      on: 1,
      listeners: 1,
      emit: 1,
      prototype: 1
    },
    Types: {
      String: [Functio

# Conectando a base de dados usando mongoose

In [3]:
mongoose.connect('mongodb://localhost/effective_store', { useUnifiedTopology: true, useNewUrlParser: true }).then(() => console.log('Connected'));
console.log('Connecting...')

Connecting...
Connected


# Definindo os países disponíveis

In [4]:
const availableCountries = _.filter(countryData.countries.all, countrie => countrie.status === 'assigned');

# Definindo o schema de Invoice

In [5]:
const invoiceSchema = new mongoose.Schema({
    invoiceNumber: String,
    dueDate: Date,
    status: String,
    totalAmount: Number,
    invoiceDoc: [{
        vendorUniqueCode: String,
        docNumber: String,
        productList: Array
    }]
})

# Definindo o schema de Sellers

In [6]:
const sellerSchema = new mongoose.Schema({
  code: {
    type: Number,
    unique: true
  },
  name: String,
  country: {
    code: { type: String, enum: availableCountries.map(country => country.alpha2) },
    name: { type: String, enum: availableCountries.map(country => country.name) },
    emojiFlag: { type: String, enum: availableCountries.map(country => country.emoji) }
  },
  invoice: [{
      invoiceNumber: String
  }],
  createdAt: Date,
  productCatalog: [{
    name: String,
    price: Number
  }]
});

# Definindo o model de Seller

In [7]:
const Seller = mongoose.model('Seller', sellerSchema);

# Gerando dados de sellers

In [ ]:
const SELLERS_LENGTH = 500;
const PRODUCTS_LENGTH = 100;

const availableProducts = _.range(PRODUCTS_LENGTH).map(() => ({
  name: faker.commerce.product(),
  price: parseFloat(faker.commerce.price())
}));

const sellers = _.range(SELLERS_LENGTH).map((sellerIndex) => {
  const productCatalogSize = _.random(1, PRODUCTS_LENGTH)
  const country = _.sample(availableCountries)
  const productCatalog = []
  _.range(productCatalogSize).forEach(() => {
    const productSample = _.sample(availableProducts)
    if (!productCatalog.includes(productSample)) {
      productCatalog.push(productSample)
    }
  });
  
  return new Seller({
    code: sellerIndex,
    name: faker.company.companyName(),
    country: { code: country.alpha2, name: country.name, emojiFlag: country.emoji },
    createdAt: faker.date.past(10),
    productCatalog
  });
});

# Populando base de dados

In [ ]:
Seller.deleteMany({}, () => console.log('cleared'));
Seller.insertMany(sellers, () => console.log('Inserted'));
console.log('Processing...')

# Query 1 - Lojistas (Sellers) com maior número de produtos disponíveis na plataforma

In [ ]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      productCatalogSize: { $cond: { if: { $isArray: "$productCatalog" }, then: { $size: "$productCatalog" }, else: "NA"} }
    }
  },
  {
    $sort : { productCatalogSize : -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 2 - Produtos mais comuns entre os lojistas associados

In [ ]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 3 - Países com mais logistas

In [ ]:
Seller.aggregate([
  {
    $group: {
      _id: '$country.name',
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { counts: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 4 - Produtos com mais lojistas por região

In [ ]:
Seller.aggregate([
  {
    $unwind: '$productCatalog'
  },
  {
    $group: {
      _id: '$productCatalog.name',
      country: { $first : "$country.name" },
      counts: { $sum: 1 }
    }
  },
  {
    $sort: { country: -1 }
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 5 - Lista dos logistas (Sellers) ordenada pelos mais novos

In [ ]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      createdAt: 1
    }
  },
  {
    $sort: { createdAt: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')

# Query 6 - Lista dos logistas (Sellers) ordenada pelos mais antiga

In [ ]:
Seller.aggregate([
  {
    $project: {
      _id: 0,
      name: 1,
      createdAt: 1
    }
  },
  {
    $sort: { createdAt: -1 }
  },
  {
    $limit: 10
  }
], (errors, result) => console.table(result))
console.log('Processing...')